# Introduction

**This notebook reads a directory of pdf files, extracts the texts using an OCR package (pytesseract), finetunes an OpenAI Curie model, and runs the model on a subdirectory of pdf files. <br><br>The structure of the notebook goes like this:**
1. [Setup](#Setup)
2. [OCR Text Extraction](#OCR-Text-Extraction)
3. [Training Data Preparation](#Training-Data-Preparation)
4. [Fine Tuning OpenAI's Curie Model](#Fine-Tuning-OpenAI's-Curie-Model)
5. [Validation Data Preparation](#Validation-Data-Preparation)
6. [Running Fine Tuned Model](#Running-Fine-Tuned-Model)

# Setup

In [1]:
from pdf2image import convert_from_bytes
import pytesseract
import os
import pandas as pd
from tqdm import tqdm
import re
import openai
import tiktoken
import random
import json
import fitz
import time
import requests
from requests.packages.urllib3.util import ssl_
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

In [2]:
os.getcwd()

'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\03-CODE\\Base computadoras'

**Set own root directory.**

In [3]:
root = r'C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE'
os.chdir(root)

**We also define other important directories.**

In [4]:
data_raw = root + r'\01-DATA_PERU\01-DATA_RAW'
data_pro = root + r'\01-DATA_PERU\02-DATA_PROCESSED'
documentation = root + r'\01-DATA_PERU\04-DATA_DOCUMENTATION'

# PDF Document Scraping

In [5]:
OSCE_computadoras_contratos = pd.read_stata(data_pro + r'\OSCE_computadoras_contratos.dta', convert_dates=True, convert_categoricals=True, index_col=None, convert_missing=False, preserve_dtypes=True, columns=None, order_categoricals=True, chunksize=None, iterator=False, compression='infer', storage_options=None)
OSCE_computadoras_contratos

,codigoconvocatoria,n_cod_contrato,urlcontrato,fecha_suscripcion_contrato,year_suscripcion,n_item1,ruc_proveedor1,ruc_destinatario_pago1,n_item2,ruc_proveedor2,ruc_destinatario_pago2,n_item3,ruc_proveedor3,ruc_destinatario_pago3,n_item4,ruc_proveedor4,ruc_destinatario_pago4
0,592583,2019652,http://zonasegura.seace.gob.pe/documentos//srv...,2020-06-29,2020.0,1,20521741555,20521741555,NaN,,,NaN,,,NaN,,
1,501339,2129753,http://zonasegura.seace.gob.pe/documentos//srv...,2019-01-11,2019.0,1,C0003151303,20601090806,NaN,,,NaN,,,NaN,,
2,480093,1181709,http://zonasegura.seace.gob.pe/documentos/mon\...,2018-10-11,2018.0,1,20601717396,20601717396,NaN,,,NaN,,,NaN,,
3,499629,1196905,http://zonasegura.seace.gob.pe/documentos/mon\...,2018-12-13,2018.0,1,20519845017,20519845017,NaN,,,NaN,,,NaN,,
4,486723,1192473,http://zonasegura.seace.gob.pe/documentos/mon\...,2018-11-21,2018.0,1,20369236874,20369236874,NaN,,,NaN,,,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,876694,2165662,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2022-12-29,2022.0,1,20606211687,20606211687,NaN,,,NaN,,,NaN,,
207,839173,2141045,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2022-09-07,2022.0,1,20452458536,20452458536,NaN,,,NaN,,,NaN,,
208,800859,2113493,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2022-05-09,2022.0,1,20519984211,20519984211,NaN,,,NaN,,,NaN,,
209,654394,2030537,https://prodapp2.seace.gob.pe/portalseace-uiwd...,2020-12-11,2020.0,1,20518446372,20518446372,NaN,,,NaN,,,NaN,,


In [6]:
save_directory = documentation + r'\Computer_sample\downloaded_pdfs'

# Create the directory if it doesn't exist
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [9]:
# Override SSL settings
# ssl_.DEFAULT_CIPHERS += ':HIGH:!DH:!aNULL'

# Disable only DH cipher
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS = 'HIGH:!DH:!aNULL'

failed_downloads_df = pd.DataFrame(columns=['n_cod_contrato', 'urlcontrato', 'failed_download'])

# Download and save PDFs
for index, row in tqdm(OSCE_computadoras_contratos.iterrows(), total=OSCE_computadoras_contratos.shape[0]):
    url = row['urlcontrato']
    contract_code = row['n_cod_contrato']
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=InsecureRequestWarning)
            response = requests.get(url, verify=False)
            if response.status_code == 200:
                # Generate a name for the PDF based on the contract's code
                filename = os.path.join(save_directory, f'pdf_{contract_code}.pdf')
                with open(filename, 'wb') as f:
                    f.write(response.content)
            else:
                failed_downloads_df = pd.concat([failed_downloads_df,pd.DataFrame({'n_cod_contrato': contract_code, 'urlcontrato': url, 'failed_download': 1}, index=[0])], ignore_index=True)
                #failed_downloads_df = failed_downloads_df.append({'Contract Code': contract_code, 'URL': url}, ignore_index=True)
                print(f'Failed to download PDF {contract_code} from {url}. Status code: {response.status_code}')
    except requests.exceptions.SSLError as e:
        print(f"An SSL error occurred: {e} in contract {contract_code} with url: {url}")
        
    except Exception as e:
        print(f"An unexpected error occurred: {e} in contract {contract_code} with url: {url}")

  0%|▍                                                                                 | 1/211 [00:02<07:11,  2.05s/it]

An unexpected error occurred: HTTPConnectionPool(host='zonasegura.seace.gob.pe', port=80): Max retries exceeded with url: /documentos//srv/nfs4/contratos/0e93aa44-0cd8-4898-a697-0308fa81bad0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000246A5E31870>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')) in contract 2019652 with url: http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/0e93aa44-0cd8-4898-a697-0308fa81bad0



  1%|▊                                                                                 | 2/211 [00:04<07:14,  2.08s/it]

An unexpected error occurred: HTTPConnectionPool(host='zonasegura.seace.gob.pe', port=80): Max retries exceeded with url: /documentos//srv/nfs4/contratos/f486d0d2-a8b3-4725-b950-b67876b6526d (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000246A5E32CB0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')) in contract 2129753 with url: http://zonasegura.seace.gob.pe/documentos//srv/nfs4/contratos/f486d0d2-a8b3-4725-b950-b67876b6526d



  1%|█▏                                                                                | 3/211 [00:06<07:11,  2.08s/it]

...


 45%|████████████████████████████████████▊                                            | 96/211 [03:29<04:10,  2.17s/it]

An unexpected error occurred: HTTPConnectionPool(host='zonasegura.seace.gob.pe', port=80): Max retries exceeded with url: /documentos/mon%5Cdocs%5Ccontratos%5C2020%5C24%5C362546717012020172439.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000246A5E319C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')) in contract 1268446 with url: http://zonasegura.seace.gob.pe/documentos/mon\docs\contratos\2020\24\362546717012020172439.pdf



 46%|█████████████████████████████████████▏                                           | 97/211 [03:31<04:06,  2.16s/it]

An unexpected error occurred: HTTPConnectionPool(host='zonasegura.seace.gob.pe', port=80): Max retries exceeded with url: /documentos/mon%5Cdocs%5Ccontratos%5C2020%5C421%5C362935408042020110558.pdf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000246A5E30FD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')) in contract 1278024 with url: http://zonasegura.seace.gob.pe/documentos/mon\docs\contratos\2020\421\362935408042020110558.pdf


100%|████████████████████████████████████████████████████████████████████████████████| 211/211 [07:01<00:00,  2.00s/it]


In [10]:
failed_downloads_df.drop('urlcontrato',
  axis='columns', inplace=True)

# OCR Text Extraction

**OCR extraction with Tesseract in Windows requires to have the program installed from this [link](https://github.com/UB-Mannheim/tesseract/wiki).<br> After installing, we need to specify the location of the exe file as below.**

In [11]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

**We first define the directory where the pdfs are located.**

In [12]:
pdfs = documentation + r'\Computer_sample\downloaded_pdfs'

**We span through the files of the base pdf directory, and its subdirectories, storing the filepath of each document.**

In [13]:
filenames = []

for base, dirs, files in os.walk(pdfs):
    for filename in files:
        if (filename.lower().endswith('.pdf')):
            filenames.append(os.path.join(base, filename))

random.choices(filenames, k=5)

['C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Computer_sample\\downloaded_pdfs\\pdf_2119932.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Computer_sample\\downloaded_pdfs\\pdf_2162463.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Computer_sample\\downloaded_pdfs\\pdf_2164720.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Computer_sample\\downloaded_pdfs\\pdf_2147231.pdf',
 'C:\\Users\\matia\\OneDrive - Universidad del Pacífico\\01-Medidas_emergencia_PE\\01-DATA_PERU\\04-DATA_DOCUMENTATION\\Computer_sample\\downloaded_pdfs\\pdf_2168525.pdf']

**We open each path and extract the text using the fitz package (a simple pdf reader). <br>Then we identify if the pdf was a scanned document (an image without *"highlight-able"* text) by setting a length threshold. <br>Only if the read pdf's text has a length below the threshold, we proceed by re-extracting the text using the Tesseract OCR package. <br>Each text is stored in a dataframe containing its filename, text, extraction type and file id.**

In [14]:
# Create a dataframe to store the texts of each PDF
pdf_texts_df = pd.DataFrame(columns = ['filename', 'text', 'extraction_type'])
broken_pdfs_df = pd.DataFrame(columns=['n_cod_contrato', 'broken_pdf'])

# Loop through every file in the directory
for filename in tqdm(filenames):
    if filename.lower().endswith('.pdf'):
        try:
        
            # Read the text directly from the PDF file
            reader = fitz.open(filename)
            pdf_text = ''

            for page in reader:
                pdf_text+=page.get_text()+' '
            
            pdf_texts_df.loc[filenames.index(filename), 'extraction_type'] = 'PDF_Reader'
            
            if len(re.sub(r'[^a-zA-Z]', '', pdf_text))<1000:
            
                # Open the PDF file
                with open(filename, 'rb') as file:
                    pdf_bytes = file.read()

                # Convert the PDF to images
                images = convert_from_bytes(pdf_bytes)

                # Use OCR to extract text from each image/page
                pdf_text = ''
                for i, image in enumerate(images):
                    text = pytesseract.image_to_string(image)
                    pdf_text+=text+' '
            
                pdf_texts_df.loc[filenames.index(filename), 'extraction_type'] = 'OCR'

            # Clean the extracted text

            clean_text = re.sub('\$+', ' ', pdf_text)  # Replace multiple \$ with a space
            clean_text = re.sub('\n+', ' ', clean_text)  # Replace multiple newlines with one space
            clean_text = re.sub('\.+', '.', clean_text)  # Replace multiple . with one space
            clean_text = re.sub('\,+', ',', clean_text)  # Replace multiple newlines with one space
            clean_text = clean_text.replace(';', ' ')  # Replace semicolons with spaces
            clean_text = re.sub(' +', ' ', clean_text)  # Replace multiple spaces with one
            clean_text = re.sub(r'[^a-zA-ZÀ-ÿ0-9 \,\.\/\:]', '', clean_text)
        
            # Store the joined text in the dataframe, using the filename (without .pdf) as the key
            pdf_texts_df.loc[filenames.index(filename), 'filename'] = filename[:-4]
            pdf_texts_df.loc[filenames.index(filename), 'text'] = clean_text
            
        except Exception as e:
            error_message = str(e)
            if 'cannot open broken document' in error_message:  # Replace 'FileDataError' with the actual error message you expect
                print(f"A FileDataError occurred: {e} in {filename}")
                broken_pdfs_df = pd.concat([broken_pdfs_df, pd.DataFrame({'n_cod_contrato': [filename.replace(pdfs+'\\pdf_', '').replace('.pdf','')], 'broken_pdf': 1}, index=[0])], ignore_index=True)
                #broken_pdfs_df = pd.concat([broken_pdfs_df, pd.DataFrame({'Contract Code': filename.str.replace(pdfs+'\\pdf_', '', regex=False)}, index=[0])], ignore_index=True)
            else:
                print(f"An unspecified error occurred: {e} in {filename}")

 71%|█████████████████████████████████████████████████████████▌                       | 81/114 [20:26<05:22,  9.78s/it]

An unspecified error occurred: Image size (274270563 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack. in C:\Users\matia\OneDrive - Universidad del Pacífico\01-Medidas_emergencia_PE\01-DATA_PERU\04-DATA_DOCUMENTATION\Computer_sample\downloaded_pdfs\pdf_2153720.pdf


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [31:46<00:00, 16.72s/it]


In [15]:
pdf_texts_df['n_cod_contrato'] = pdf_texts_df['filename'].str.replace(pdfs+'\\pdf_', '', regex=False)
pdf_texts_df.drop('filename', axis='columns', inplace=True)

In [16]:
computer_analysis_dfs = data_pro + r'\computer_analysis_dfs'

# Create the directory if it doesn't exist
if not os.path.exists(computer_analysis_dfs):
    os.makedirs(computer_analysis_dfs)

In [17]:
display(failed_downloads_df)
display(broken_pdfs_df)
display(pdf_texts_df)

,n_cod_contrato,failed_download


,n_cod_contrato,broken_pdf


,text,extraction_type,n_cod_contrato
0,RUC N 20187651361 JR. MIRAFLORES NRO. SIN LA L...,PDF_Reader,2000640
1,PERU Ministerio Instituto i del Ambiente Geof...,OCR,2003634
2,CONTRATO N 172020MDS ADQUISICION DE MOBILIARIO...,OCR,2010690
3,CONTRATO N 182020MDS ADQUISICION DE MOBILIARIO...,OCR,2010704
4,MUNICIPALIDAD DISTRITAL DESONDORIELO IREGION/P...,OCR,2010706
...,...,...,...
109,Universidad Nacional Hermilio Valdizan de Huan...,OCR,2175999
110,126 ORDEN DE COMPRA GUIA DE ois nes aio Sef...,OCR,2184078
111,Sistema Integrado de Gestién Administrativa Mé...,OCR,2184774
112,MUNICIPALIDAD DISTRITAL DE ANCHONGA as a sae e...,OCR,2186739


In [18]:
pdf_texts_df.to_excel(computer_analysis_dfs + r'\pdf_texts.xlsx', index = False)
failed_downloads_df.to_excel(computer_analysis_dfs + r'\failed_downloads.xlsx', index = False)
broken_pdfs_df.to_excel(computer_analysis_dfs + r'\broken_pdfs.xlsx', index = False)

# Preparing Data for Unit Price Extraction

In [27]:
extraction_df = pd.read_excel(data_pro + r'\computer_analysis_dfs\pdf_texts.xlsx')

In [28]:
extraction_df

,text,extraction_type,n_cod_contrato
0,RUC N 20187651361 JR. MIRAFLORES NRO. SIN LA L...,PDF_Reader,2000640.0
1,PERU Ministerio Instituto i del Ambiente Geof...,OCR,2003634.0
2,CONTRATO N 172020MDS ADQUISICION DE MOBILIARIO...,OCR,2010690.0
3,CONTRATO N 182020MDS ADQUISICION DE MOBILIARIO...,OCR,2010704.0
4,MUNICIPALIDAD DISTRITAL DESONDORIELO IREGION/P...,OCR,2010706.0
...,...,...,...
109,Universidad Nacional Hermilio Valdizan de Huan...,OCR,2175999.0
110,126 ORDEN DE COMPRA GUIA DE ois nes aio Sef...,OCR,2184078.0
111,Sistema Integrado de Gestién Administrativa Mé...,OCR,2184774.0
112,MUNICIPALIDAD DISTRITAL DE ANCHONGA as a sae e...,OCR,2186739.0


In [29]:
extraction_df['text'] = extraction_df['text'].astype(str)
extraction_df['text'] = extraction_df['text'].str.lower()
extraction_df['text'] = extraction_df['text'].apply(lambda x: re.sub('\s+', ' ', x).strip())
extraction_df['text'] = extraction_df['text'].apply(lambda x: re.sub(r'(\.|\,)\1+', r'\1', x).strip())
extraction_df.drop('extraction_type', axis='columns', inplace=True)

In [30]:
# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    tokens = tokenizer.encode(text)
    """Yield successive n-sized chunks from text."""
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.9 * n and 1.1 * n tokens
        j = min(i + int(1.1 * n), len(tokens))
        while j > i + int(0.9 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.9 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j

In [31]:
# Initialise tokenizer
tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')

prompt=[]

for i in tqdm(range(len(extraction_df['text']))):
    chunks = create_chunks(extraction_df.loc[i,'text'], 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    if len(text_chunks)>=2:
        prompt.append(' '.join(text_chunks[:1]+text_chunks[-1:]))
    else:
        prompt.append(' '.join(text_chunks))
extraction_df['text'] = prompt

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [00:01<00:00, 72.59it/s]


In [32]:
extraction_df['text'] = extraction_df['text'] + '---->'

In [33]:
os.environ['OPENAI_API_KEY'] = ""

In [34]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [35]:
# Iterate through each row in DataFrame
for index, row in tqdm(extraction_df.iterrows(), total=extraction_df.shape[0]):
    user_content = row['text']
    time.sleep(2) 
    
    try:
        # Call OpenAI API
        completion = openai.ChatCompletion.create(
            model="ft:gpt-3.5-turbo-0613:personal::7wfSDkB5",
            request_timeout = 100,
            messages=[
                {"role": "system", "content": "Dado el texto extraido de un contrato, extrae los precios unitarios de los bienes comprados."},
                {"role": "user", "content": user_content}
            ]
        )
    
        # Extract the generated message and store it in DataFrame
        generated_message = completion.choices[0].message['content']
        extraction_df.at[index, 'gpt_unit_prices'] = generated_message

    except openai.error.RateLimitError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 1
        print(f"Rate limit error. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        
    except openai.error.ServiceUnavailableError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 1
        print(f"Service Unavailable error. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)

    except openai.error.APIError as e:
        retry_time = e.retry_after if hasattr(e, 'retry_after') else 1
        print(f"API error occurred. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)

    except OSError as e:
        retry_time = 1  # Adjust the retry time as needed
        print(f"Connection error occurred: {e}. Retrying in {retry_time} seconds...")      
        time.sleep(retry_time)
        
    except requests.Timeout as e:
        retry_time = 1  # Adjust the retry time as needed
        print(f"Timeout error occurred: {e}. Retrying in {retry_time} seconds...")      
        time.sleep(retry_time)
        
    except Exception as e:
        retry_time = 1
        print(f"An unexpected error occurred: {e}. Retrying in {retry_time} seconds...")
        time.sleep(retry_time)
        
# Print the updated DataFrame to check if it worked
extraction_df

 11%|████████▌                                                                        | 12/114 [00:44<05:34,  3.28s/it]

Service Unavailable error. Retrying in 1 seconds...


 46%|████████████████████████████████████▉                                            | 52/114 [03:10<02:42,  2.63s/it]

Service Unavailable error. Retrying in 1 seconds...


100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [06:52<00:00,  3.62s/it]


,text,n_cod_contrato,gpt_unit_prices
0,ruc n 20187651361 jr. miraflores nro. sin la l...,2000640.0,3300.0 \n\n###\n\n
1,peru ministerio instituto i del ambiente geofi...,2003634.0,7681.444 \n\n###\n\n
2,contrato n 172020mds adquisicion de mobiliario...,2010690.0,\n\n###\n\n
3,contrato n 182020mds adquisicion de mobiliario...,2010704.0,\n\n###\n\n
4,municipalidad distrital desondorielo iregion/p...,2010706.0,\n\n###\n\n
...,...,...,...
109,universidad nacional hermilio valdizan de huan...,2175999.0,\n\n###\n\n
110,126 orden de compra guia de ois nes aio seftor...,2184078.0,10535.0; 7499.0; 8789.0 \n\n###\n\n
111,sistema integrado de gestién administrativa mé...,2184774.0,6060.0 \n\n###\n\n
112,municipalidad distrital de anchonga as a sae e...,2186739.0,3000.0; 7450.0; 2000.0; 3900.0; 980.0; 6400.0...


In [36]:
extraction_df.to_excel(computer_analysis_dfs + r'\extraction_df.xlsx', index = False)